<img alt="Colaboratory logo" width="15%" src="https://raw.githubusercontent.com/carlosfab/escola-data-science/master/img/novo_logo_bg_claro.png">

#### **Data Science na Prática**
*by [sigmoidal.ai](https://sigmoidal.ai)*

---

# Workflow para Machine Learning

<center><img src="https://images.unsplash.com/photo-1533749871411-5e21e14bcc7d?ixid=MnwxMjA3fDB8MHxwaG90by1wYWdlfHx8fGVufDB8fHx8&ixlib=rb-1.2.1&auto=format&fit=crop&w=1471&q=80" width="60%"></center>

O *workflow* para problemas de Machine Learning apresentado pelo Google, é composto de 6 etapas principais:

1. Adquirir os Dados
2. Explorar os Dados
3. Preparar os Dados
4. Construir, Treinar e Avaliar seu Modelo
5. Otimizar os Hiperparâmetros
6. Deploy do Modelo

Há ainda uma etapa `2.5 Escolher o Modelo`, que não faz parte dos *frameworks* tradicionais, mas que merece destaque devido a sua importância crítica.

Para conhecer mais sobre esse workflow, visite o [guideline da Google](https://developers.google.com/machine-learning/guides/text-classification).

<center><img src="https://raw.githubusercontent.com/carlosfab/dsnp2/master/img/Workflow-2.png" width="700"></center>


## 1. Aquisição dos dados

<center><img src="https://images.unsplash.com/photo-1528582654826-585a71fcf7f4?ixid=MnwxMjA3fDB8MHxwaG90by1wYWdlfHx8fGVufDB8fHx8&ixlib=rb-1.2.1&auto=format&fit=crop&w=1470&q=80" width="50%"></center>

De onde virão os seus dados? De um `csv`, `xls`, de um banco de dados `mysql`, consumindo microserviços, fazendo web scraping ou usando APIs disponibilizadas por empresas como Twitter, RD Station e Google?

Se for por meio de APIs, quais são as regras para não ser bloqueado? O consumo de dados é gratuito? Precisa criar uma chave?

Se for por meio das planilhas da sua empresa, você sabe se elas têm dependências, se alguém pode alterar a lógica ou corromper o arquivo? Os lançamentos são manuais, por meio de campos abertos, ou fechados?

Os dados estão minimamente balanceados?

As informações representam estatisticamente todas as situações possíveis?

Depois de importar os dados, você realizou cheques para ver se eles estão consistentes?

**Exemplo 1**

In [ ]:
def load_imdb_sentiment_analysis_dataset(data_path, seed=123):
    """Loads the IMDb movie reviews sentiment analysis dataset.

    # Arguments
        data_path: string, path to the data directory.
        seed: int, seed for randomizer.

    # Returns
        A tuple of training and validation data.
        Number of training samples: 25000
        Number of test samples: 25000
        Number of categories: 2 (0 - negative, 1 - positive)

    # References
        Mass et al., http://www.aclweb.org/anthology/P11-1015

        Download and uncompress archive from:
        http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
    """
    imdb_data_path = os.path.join(data_path, 'aclImdb')

    # Load the training data
    train_texts = []
    train_labels = []
    for category in ['pos', 'neg']:
        train_path = os.path.join(imdb_data_path, 'train', category)
        for fname in sorted(os.listdir(train_path)):
            if fname.endswith('.txt'):
                with open(os.path.join(train_path, fname)) as f:
                    train_texts.append(f.read())
                train_labels.append(0 if category == 'neg' else 1)

    # Load the validation data.
    test_texts = []
    test_labels = []
    for category in ['pos', 'neg']:
        test_path = os.path.join(imdb_data_path, 'test', category)
        for fname in sorted(os.listdir(test_path)):
            if fname.endswith('.txt'):
                with open(os.path.join(test_path, fname)) as f:
                    test_texts.append(f.read())
                test_labels.append(0 if category == 'neg' else 1)

    # Shuffle the training data and labels.
    random.seed(seed)
    random.shuffle(train_texts)
    random.seed(seed)
    random.shuffle(train_labels)

    return ((train_texts, np.array(train_labels)),
            (test_texts, np.array(test_labels)))

**Exemplo 2**

In [ ]:
# import the necessary packages
import numpy as np
import cv2
import os

class SimpleDatasetLoader:
	def __init__(self, preprocessors=None):
		# store the image preprocessor
		self.preprocessors = preprocessors

		# if the preprocessors are None, initialize them as an
		# empty list
		if self.preprocessors is None:
			self.preprocessors = []

	def load(self, imagePaths, verbose=-1):
		# initialize the list of features and labels
		data = []
		labels = []

		# loop over the input images
		for (i, imagePath) in enumerate(imagePaths):
			# load the image and extract the class label assuming
			# that our path has the following format:
			# /path/to/dataset/{class}/{image}.jpg
			image = cv2.imread(imagePath)
			label = imagePath.split(os.path.sep)[-2]

			# check to see if our preprocessors are not None
			if self.preprocessors is not None:
				# loop over the preprocessors and apply each to
				# the image
				for p in self.preprocessors:
					image = p.preprocess(image)

			# treat our processed image as a "feature vector"
			# by updating the data list followed by the labels
			data.append(image)
			labels.append(label)

			# show an update every `verbose` images
			if verbose > 0 and i > 0 and (i + 1) % verbose == 0:
				print("[INFO] processed {}/{}".format(i + 1,
					len(imagePaths)))

		# return a tuple of the data and labels
		return (np.array(data), np.array(labels))

**Exemplo 3**

In [ ]:
# USAGE
# python build_lisa_records.py

# import the necessary packages
from config import lisa_config as config
from pyimagesearch.utils.tfannotation import TFAnnotation
from sklearn.model_selection import train_test_split
from PIL import Image
import tensorflow as tf
import os

def main(_):
	# open the classes output file
	f = open(config.CLASSES_FILE, "w")

	# loop over the classes
	for (k, v) in config.CLASSES.items():
		# construct the class information and write to file
		item = ("item {\n"
				"\tid: " + str(v) + "\n"
				"\tname: '" + k + "'\n"
				"}\n")
		f.write(item)

	# close the output classes file
	f.close()

	# initialize a data dictionary used to map each image filename
	# to all bounding boxes associated with the image, then load
	# the contents of the annotations file
	D = {}
	rows = open(config.ANNOT_PATH).read().strip().split("\n")

	# loop over the individual rows, skipping the header
	for row in rows[1:]:
		# break the row into components
		row = row.split(",")[0].split(";")
		(imagePath, label, startX, startY, endX, endY, _) = row
		(startX, startY) = (float(startX), float(startY))
		(endX, endY) = (float(endX), float(endY))

		# if we are not interested in the label, ignore it
		if label not in config.CLASSES:
			continue

		# build the path to the input image, then grab any other
		# bounding boxes + labels associated with the image
		# path, labels, and bounding box lists, respectively
		p = os.path.sep.join([config.BASE_PATH, imagePath])
		b = D.get(p, [])

		# build a tuple consisting of the label and bounding box,
		# then update the list and store it in the dictionary
		b.append((label, (startX, startY, endX, endY)))
		D[p] = b

	# create training and testing splits from our data dictionary
	(trainKeys, testKeys) = train_test_split(list(D.keys()),
		test_size=config.TEST_SIZE, random_state=42)

	# initialize the data split files
	datasets = [
		("train", trainKeys, config.TRAIN_RECORD),
		("test", testKeys, config.TEST_RECORD)
	]

	# loop over the datasets
	for (dType, keys, outputPath) in datasets:
		# initialize the TensorFlow writer and initialize the total
		# number of examples written to file
		print("[INFO] processing '{}'...".format(dType))
		writer = tf.python_io.TFRecordWriter(outputPath)
		total = 0

		# loop over all the keys in the current set
		for k in keys:
			# load the input image from disk as a TensorFlow object
			encoded = tf.gfile.GFile(k, "rb").read()
			encoded = bytes(encoded)

			# load the image from disk again, this time as a PIL
			# object
			pilImage = Image.open(k)
			(w, h) = pilImage.size[:2]

			# parse the filename and encoding from the input path
			filename = k.split(os.path.sep)[-1]
			encoding = filename[filename.rfind(".") + 1:]

			# initialize the annotation object used to store
			# information regarding the bounding box + labels
			tfAnnot = TFAnnotation()
			tfAnnot.image = encoded
			tfAnnot.encoding = encoding
			tfAnnot.filename = filename
			tfAnnot.width = w
			tfAnnot.height = h

			# loop over the bounding boxes + labels associated with
			# the image
			for (label, (startX, startY, endX, endY)) in D[k]:
				# TensorFlow assumes all bounding boxes are in the
				# range [0, 1] so we need to scale them
				xMin = startX / w
				xMax = endX / w
				yMin = startY / h
				yMax = endY / h

				# update the bounding boxes + labels lists
				tfAnnot.xMins.append(xMin)
				tfAnnot.xMaxs.append(xMax)
				tfAnnot.yMins.append(yMin)
				tfAnnot.yMaxs.append(yMax)
				tfAnnot.textLabels.append(label.encode("utf8"))
				tfAnnot.classes.append(config.CLASSES[label])
				tfAnnot.difficult.append(0)

				# increment the total number of examples
				total += 1

			# encode the data point attributes using the TensorFlow
			# helper functions
			features = tf.train.Features(feature=tfAnnot.build())
			example = tf.train.Example(features=features)

			# add the example to the writer
			writer.write(example.SerializeToString())

		# close the writer and print diagnostic information to the
		# user
		writer.close()
		print("[INFO] {} examples saved for '{}'".format(total,
			dType))

# check to see if the main thread should be started
if __name__ == "__main__":
	tf.app.run()

## 2. Explorar os dados

<center><img src="https://images.unsplash.com/photo-1614314266357-8a2e58059af5?ixid=MnwxMjA3fDB8MHxwaG90by1wYWdlfHx8fGVufDB8fHx8&ixlib=rb-1.2.1&auto=format&fit=crop&w=1021&q=80" width="50%"></center>

Entender os dados é a fase que permite você entender qual a melhor abordagem, melhor modelo, melhores features para serem usadas.

Todos os insights devem ser anotados e documentados da melhor maneira possível. Alguns questionamentos a serem feitos são:

* Qual o número de amostras?
* Qual o número total de classes?
* O dataset está balanceado?
* Quais variáveis são categóricas ou numéricas?
* Qual a porcentagem de valores ausentes?
* Há outliers presentes?
* Qual o tipo de distribuição das suas variáveis?
* Qual a variável alvo?
* Existe correlação entre as variáveis?
* Quais transformações são aplicáveis, começando a raciocinar com o nosso modelo?

## 2.5 Escolher o Modelo

<center><img src="https://www.incimages.com/uploaded_files/image/1920x1080/getty_178784049_9706479704500197_55273.jpg" width="60%"></center>


Baseado nas observações e métricas identificadas nas etapas anteriores, você começa a ter uma ideia de qual modelo seria mais adequado ao problema que você está atacando.

Obviamente, depende muito da experiência e conhecimento, mas existem alguns guias que ajudam na escolha do algoritmo.

Veja o material que está em anexo na aula.

# 3 Preparação dos Dados

<center><img src="https://images.unsplash.com/photo-1571666521805-f5e8423aba9d?ixid=MnwxMjA3fDB8MHxwaG90by1wYWdlfHx8fGVufDB8fHx8&ixlib=rb-1.2.1&auto=format&fit=crop&w=1470&q=80" width="55%"></center>

Existem várias transformações que podem ser possíveis, mas às vezes fica confuso quais devemos fazer. Gosto de dividir entre transformações obrigatórias e transformações opcionais:

1. Transformações obrigatórias
    * Converter dados não numéricos em numéricos, pois você não vai conseguir realizar operações vetorizadas sem converter strings, por exemplo, em variáveis numéricas.
    * Redimensionar inputs para valores fixos, pois o seu modelo espera ser alimentado sempre da mesma maneira.
    * Lidar com os valores ausentes, seja excluindo as entradas ou substituindo seus valores
    * Remoção de outliers, uma vez que eles irão dar um viés ao nosso modelo.
2. Transformações opcionais
    * Normalizar ou padronizar as features, dependendo do modelo e demandas específicas.
    * Tratamento adequado de strings, como deixar todos os caracteres em minúsculo.
    * Criar um fluxo para deixar o dataset sempre anônimo.
    * Reduzir a dimensionalidade do problema, dependendo da quantidade e qualidade das suas features.
    * Criar novas variáveis, buscando técnicas de feature engineering.

Os dados podem ser preparados antes ou dentro do modelo. Veja as vantagens e desvantagens de cada uma dessas abordagens [neste link](https://developers.google.com/machine-learning/data-prep/transform/introduction).

# 4 Construir, Treinar e Avaliar o seu modelo

<center><img src="https://images.unsplash.com/photo-1587654780291-39c9404d746b?ixid=MnwxMjA3fDB8MHxwaG90by1wYWdlfHx8fGVufDB8fHx8&ixlib=rb-1.2.1&auto=format&fit=crop&w=1470&q=80" width="55%"></center>

Você pode construir um modelo baseline, ou seja, um modelo bem básico para ser o seu parâmetro de desempenho.

É comum muitos cientistas de dados pegarem apenas as variáveis numéricas e não se preocuparem com feature engineering ou codificação de variáveis categóricas. Como eu disse, um baseline é opcional, mas muito bem-vindo.

Você deve entender as melhores métricas desde já, para realizar testes com vários modelos diferentes. O desempenho deles pode ser inferido pelo cross-validation.

O modelo de melhor desempenho, de acordo com o cross-validation, será escolhido e poderemos melhorar ainda mais o desempenho dele.

Detalhe, esse modelo final deve ser treinado em cima do dataset de treino completo. Como ele ainda não teve contato com os dados de teste, a avaliação final será feita com eles.

# 5 Otimizar os Hiperparâmetros

<center><img src="https://images.unsplash.com/photo-1628666173519-66351692f747?ixid=MnwxMjA3fDB8MHxwaG90by1wYWdlfHx8fGVufDB8fHx8&ixlib=rb-1.2.1&auto=format&fit=crop&w=1074&q=80" width="55%"></center>


Cada modelo tem a sua especificidade. Você deve conhecer cada parâmetro e o quanto ele impacta no desempenho e nas variáveis.

Um modelo de Regressão Logística, por exemplo, tem muito pouca coisa a ser feito nesta etapa. Já um modelo classificador XGBoost, precisa que você siga determinadas etapas de otimização, a fim de não resultar em overfitting ou piora no desempenho.

Hyperparameters tunning é uma arte, porém com método. Não use os dados de teste e trabalhe com Grid Search.

# 6 Deploy do Modelo

<center><img src="https://blog-geek-midia.s3.amazonaws.com/wp-content/uploads/2020/11/11154014/o-que-e-deploy-e1605120444248.png" width="55%"></center>


Deploy significar colocar o modelo em produção. Isso pode ser tão simples como disponibilizar uma API para ser consumida (que retorna as previsões do seu modelo), ou um sistema completo.

Tenha em mente os seguintes pontos, importantes para o deploy:

* Os seus novos dados devem ter a mesma distribuição, aquele usado nos dados de treino e validação.
* Reveja o seu pipeline e identifique quais pré-processamentos foram feitos.
* Reavalie o modelo constantemente. Decida se é necessário retreinar. Caso seja, mude a distribuição que será aplicada.